### we will use two sources:
1. pycountry
2. nooa country list 

In [2]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

#### source 1

In [ ]:
conn.execute('''
CREATE TABLE IF NOT EXISTS countries (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    alpha_2 TEXT NOT NULL
)
''')
conn.commit()

In [ ]:
import pycountry
for country in pycountry.countries:
    conn.execute('''
    INSERT INTO countries (name, alpha_2) VALUES (?, ?)
    ''', (country.name, country.alpha_2))

#### source 2

In [3]:
# Fetch the data from the NOAA link
url = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt'
response = requests.get(url)
data = response.text

# Create the table in the database
conn.execute('''
CREATE TABLE IF NOT EXISTS noaa_countries (
    code TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
''')

# Insert the data into the table
for line in data.splitlines():
    if line.strip():
        code = line[:2].strip()
        name = line[3:].strip()
        conn.execute('''
        INSERT OR IGNORE INTO noaa_countries (code, name) VALUES (?, ?)
        ''', (code, name))

conn.commit()

In [4]:
pd.read_sql_query('SELECT count(*) FROM noaa_countries', conn)

,count(*)
0,219


In [5]:
pd.read_sql_query('SELECT count(*) FROM countries', conn)

,count(*)
0,249
